# ConversationKGMemory

지식 그래프의 힘을 활용하여 정보를 저장하고 불러옵니다.

이를 통해 모델이 서로 다른 개체 간의 관계를 이해하는 데 도움을 주고, 복잡한 연결망과 역사적 맥락을 기반으로 대응하는 능력을 향상시킵니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationKGMemory

In [5]:
llm = ChatOpenAI(temperature=0)

memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context(
    {"input": "이쪽은 Pangyo 에 거주중인 김셜리씨 입니다."},
    #{"input": "이쪽은 판교에 거주중인 김셜리씨 입니다."},
    {"output": "김셜리씨는 누구시죠?"},
)
memory.save_context(
    {"input": "김셜리씨는 우리 회사의 신입 디자이너입니다."},
    {"output": "만나서 반갑습니다."},
)

In [6]:
memory.load_memory_variables({"input": "김셜리씨는 누구입니까?"})

{'history': [SystemMessage(content='On Pangyo: Pangyo has resident 김셜리씨.', additional_kwargs={}, response_metadata={}),
  SystemMessage(content='On 김셜리씨: 김셜리씨 is a 신입 디자이너. 김셜리씨 is in 우리 회사.', additional_kwargs={}, response_metadata={})]}

## Chain 에 메모리 활용하기

`ConversationChain` 에 `ConversationKGMemory` 를 메모리로 지정하여 대화를 나눈 후 memory 를 확인해 보도록 하겠습니다.


In [7]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

llm = ChatOpenAI(temperature=0)

template = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template)

conversation_with_kg = ConversationChain(
    llm=llm, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)

C:\Users\park0\AppData\Local\Temp\ipykernel_5600\4232739924.py:21: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation_with_kg = ConversationChain(


## ConversationKGMemory에서 LLM의 역할

`ConversationKGMemory(llm=llm)`에서 LLM은 다음과 같은 역할을 수행합니다:

1. **개체 추출 (Entity Extraction)**: 
   - 대화에서 중요한 개체(사람, 장소, 사물 등)를 식별
   - 예: "Teddy", "Shirley", "company", "designer"

2. **관계 추출 (Relationship Extraction)**: 
   - 개체들 간의 관계를 파악
   - 예: "Shirley is a coworker of Teddy", "Shirley is a designer at company"

3. **지식 그래프 구성**: 
   - 추출된 정보를 구조화된 지식 그래프로 변환
   - 트리플 형태로 저장: (주체, 관계, 객체)

4. **컨텍스트 기반 정보 검색**: 
   - 질문이 들어오면 관련 지식을 그래프에서 검색하여 제공

In [8]:
# ConversationKGMemory의 동작 방식 확인
llm = ChatOpenAI(temperature=0)

# 독립적인 KG Memory 생성
kg_memory = ConversationKGMemory(llm=llm)

# 대화 저장
kg_memory.save_context(
    {"input": "안녕하세요! 저는 서울에 사는 김민수입니다. 저는 소프트웨어 개발자로 일하고 있어요."},
    {"output": "안녕하세요 김민수님! 만나서 반갑습니다."}
)

kg_memory.save_context(
    {"input": "제 동료인 박지영씨도 소개해드릴게요. 박지영씨는 우리 팀의 UI/UX 디자이너입니다."},
    {"output": "박지영님도 만나서 반갑네요!"}
)

# 지식 그래프에서 추출된 정보 확인
print("=== 지식 그래프에 저장된 정보 ===")
print("KG 메모리 키:", list(kg_memory.kg.get_triples()))

# 특정 질문에 대한 관련 정보 검색
print("\n=== '김민수는 누구인가?' 질문에 대한 관련 정보 ===")
relevant_info = kg_memory.load_memory_variables({"input": "김민수는 누구인가요?"})
print(relevant_info)

=== 지식 그래프에 저장된 정보 ===
KG 메모리 키: [('Kim Min Soo', 'Seoul', 'lives in'), ('Kim Min Soo', 'software developer', 'is a'), ('Park Jiyoung', 'UI/UX designer', 'is a'), ('Park Jiyoung', 'our team', 'is in')]

=== '김민수는 누구인가?' 질문에 대한 관련 정보 ===
{'history': ''}


첫 번째 대화를 시작합니다. 간단한 인물에 대한 정보를 제공해 보겠습니다.


In [9]:
conversation_with_kg.predict(
    input="My name is Teddy. Shirley is a coworker of mine, and she's a new designer at our company."
)

"Hello Teddy! It's nice to meet you. Shirley must be a talented designer if she's new at your company. What kind of projects is she working on?"

Shirley 라는 사람에 대한 질문을 진행합니다.


In [10]:
# Shirley 에 대한 질문
conversation_with_kg.memory.load_memory_variables({"input": "who is Shirley?"})

{'history': "On Shirley: Shirley is a coworker of Teddy. Shirley is a new designer at Teddy's company."}